In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
torch.__version__

'1.8.0+cu111'

In [2]:
BATCH_SIZE = 1024 # 512 batches needs about 2G of GPU Memory
EPOCHS = 40 # total training epochs
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('mnist', train = True, download = True, 
                       transform = transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size = BATCH_SIZE, shuffle = True)

In [4]:
test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('mnist', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=BATCH_SIZE, shuffle=True)

In [5]:
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, 3),
            nn.ReLU(True),
            nn.BatchNorm2d(32),
            nn.Conv2d(32, 32, 3),
            nn.ReLU(True),
            nn.BatchNorm2d(32),
            nn.ZeroPad2d((2, 2, 2, 2)),
            nn.Conv2d(32, 32, 5, stride = 2),
            nn.ReLU(True),
            nn.BatchNorm2d(32),
            nn.Dropout2d(0.4, True)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, 3),
            nn.ReLU(True),
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 64, 3),
            nn.ReLU(True),
            nn.BatchNorm2d(64),
            nn.ZeroPad2d((2, 2, 2, 2)),
            nn.Conv2d(64, 64, 5, stride = 2),
            nn.ReLU(True),
            nn.BatchNorm2d(64),
            nn.Dropout2d(0.4, True)
        )
        self.method1 = nn.Sequential(
            nn.Flatten(),
            nn.Linear(1024, 128),
            nn.ReLU(True),
            nn.Linear(128, 10),
            nn.ReLU(True),
            nn.Dropout(0.4, True),
            nn.Softmax(dim = 1)
        )
        self.method2 = nn.Sequential(
            nn.Conv2d(64, 128, 4),
            nn.ReLU(True),
            nn.BatchNorm2d(128),
            nn.Flatten(),
            nn.Dropout(0.4, True),
            nn.Linear(128, 10),
            nn.Softmax(dim = 1)
        )

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        #out = self.method1(out)
        out = self.method2(out)
        return out

In [6]:
model = ConvNet().to(DEVICE)
optimizer = optim.Adam(model.parameters())

In [7]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        #loss = F.nll_loss(output, target)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        if (batch_idx + 1) % 20 == 0: 
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [8]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            #test_loss += F.nll_loss(output, target, reduction = 'sum').item() # add the losses of a batch together
            test_loss += F.cross_entropy(output, target, reduction = 'sum').item()
            pred = output.max(1, keepdim = True)[1] # find the maximum index
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.1f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [9]:
for epoch in range(1, EPOCHS + 1):
    train(model, DEVICE, train_loader, optimizer, epoch)
    test(model, DEVICE, test_loader)

Train Epoch: 1 [19456/60000 (32%)]	Loss: 1.608977
Train Epoch: 1 [39936/60000 (66%)]	Loss: 1.530157

Test set: Average loss: 1.4850, Accuracy: 9828/10000 (98.3%)

Train Epoch: 2 [19456/60000 (32%)]	Loss: 1.487510
Train Epoch: 2 [39936/60000 (66%)]	Loss: 1.484487

Test set: Average loss: 1.4738, Accuracy: 9899/10000 (99.0%)

Train Epoch: 3 [19456/60000 (32%)]	Loss: 1.481391
Train Epoch: 3 [39936/60000 (66%)]	Loss: 1.475387

Test set: Average loss: 1.4717, Accuracy: 9909/10000 (99.1%)

Train Epoch: 4 [19456/60000 (32%)]	Loss: 1.476254
Train Epoch: 4 [39936/60000 (66%)]	Loss: 1.473974

Test set: Average loss: 1.4698, Accuracy: 9933/10000 (99.3%)

Train Epoch: 5 [19456/60000 (32%)]	Loss: 1.472781
Train Epoch: 5 [39936/60000 (66%)]	Loss: 1.475535

Test set: Average loss: 1.4695, Accuracy: 9928/10000 (99.3%)

Train Epoch: 6 [19456/60000 (32%)]	Loss: 1.472936
Train Epoch: 6 [39936/60000 (66%)]	Loss: 1.472928

Test set: Average loss: 1.4700, Accuracy: 9928/10000 (99.3%)

Train Epoch: 7 [19456/